<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-definitions" data-toc-modified-id="Imports-and-definitions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and definitions</a></span></li><li><span><a href="#Load-tables" data-toc-modified-id="Load-tables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load tables</a></span></li><li><span><a href="#Text-preprocessing" data-toc-modified-id="Text-preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Text preprocessing</a></span></li></ul></div>

# Analysis 2

## Imports and definitions

In [1]:
%load_ext memory_profiler

In [2]:
import re
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from nlppreprocess import NLP
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))
processor = NLP()
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

## Load tables

In [3]:
path = 'output_data/'
tweet = pd.read_csv(path+'tweet.csv', lineterminator='r')

## Text preprocessing

I spent *hours* on this and then I found tweet-preprocessor which I added to the
tweet import notebook.

Here I'll only be preprocessing the *text*.

In [4]:
%%time
tweet.set_index('id', inplace=True)
# Lower the text
tweet['text']=tweet['text'].str.lower()
# Expand some negative sentiments and common suffixes
tweet['text']=tweet['text'].str.replace("can\'t","cannot")
tweet['text']=tweet['text'].str.replace("n\'t","\ not")
tweet['text']=tweet['text'].str.replace("\'d","\ had")
tweet['text']=tweet['text'].str.replace("\'ve","\ have")
tweet['text']=tweet['text'].str.replace("\'ll","\ will")
tweet['text']=tweet['text'].str.replace("\'re","\ are")

CPU times: user 9.09 s, sys: 64.6 ms, total: 9.15 s
Wall time: 9.16 s


In [5]:
%%time
# Run the preprocessor
tweet['text']=tweet['text'].apply(processor.process)

CPU times: user 17.4 s, sys: 87.9 ms, total: 17.5 s
Wall time: 17.5 s


In [6]:
%%time
# There's still noise!
tweet['text']=tweet['text'].str.replace('\ [^ia]\ ',' ')

CPU times: user 1.33 s, sys: 19.8 ms, total: 1.35 s
Wall time: 1.35 s


In [7]:
%%time
# Tokenize to obtain a list
tweet['words'] = tweet['text'].astype(str).apply(tokenizer.tokenize)

CPU times: user 53.3 s, sys: 1.55 s, total: 54.9 s
Wall time: 54.9 s


In [8]:
%%time
# Eliminate stop words
sw.add('amp')
def remove_stopwords(wordlist, stopwords=sw):
    return [word for word in wordlist if word not in stopwords and len(word)>1]
tweet['words']=tweet['words'].apply(remove_stopwords)

CPU times: user 4.61 s, sys: 44 ms, total: 4.65 s
Wall time: 4.65 s


In [9]:
# Combine words into sentences again - but now without the junk!
tweet['wordtext'] = tweet['words'].apply(" ".join)

In [10]:
tweet.head()

,created_at,retweet_count,screen_name,text,words,wordtext
id,,,,,,
877418565,2008-08-04 17:28:51,0,JohnBoozman,conference call about weekend trip iraq visit arkansas troops,"[conference, call, weekend, trip, iraq, visit, arkansas, troops]",conference call weekend trip iraq visit arkansas troops
879695803,2008-08-06 20:35:36,0,JohnBoozman,being interviewed by karn his arkansas world trade center trip,"[interviewed, karn, arkansas, world, trade, center, trip]",interviewed karn arkansas world trade center trip
880393665,2008-08-07 13:52:52,0,JohnBoozman,kwhn in fort smith that am,"[kwhn, fort, smith]",kwhn fort smith
880474266,2008-08-07 15:12:05,0,JohnBoozman,attending military purple heart ceremony va medical center in fayetteville,"[attending, military, purple, heart, ceremony, va, medical, center, fayetteville]",attending military purple heart ceremony va medical center fayetteville
880676101,2008-08-07 18:35:25,0,JohnBoozman,touring helath south hospital in fayetteville,"[touring, helath, south, hospital, fayetteville]",touring helath south hospital fayetteville


In [11]:
%%memit
tweet.to_parquet(path+'cleantweets.parquet')

peak memory: 2419.27 MiB, increment: 596.22 MiB
